# 1 - Prepare the knowledge base
This step is to find the relevant content to the question and prepare the knowledge base(context) for GenAI for reference. To index documents with a search engine. 
**Elastic search** is more preseistitent than the toy search engine(minsearch) built before, we don't need to rebuid index everytime after the virtual machine stops, saves all the data on disk.

In [1]:
# import minsearch

In [2]:
import json

In [3]:
with open('documents.json','rt') as f_in: # read json file in reading text mode as file input
    docs_raw = json.load(f_in)

In [4]:
# docs_raw

In [5]:
len(docs_raw) # means it has 3 different courses 

3

In [6]:
documents = []

for course_dict in docs_raw: 
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc) 

#### 3 steps for this loop:
# 1. Store the flattened course_dict['documents'] in each doc as a dictionary,
# 2. Add a new key'course' in each doc. 
# 3. Append doc to documents dict. 


In [7]:
documents[0:2]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'}]

In [8]:
from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
# to check if elasticsearch is working 
es_client.info()

ObjectApiResponse({'name': 'd4cf68c5d874', 'cluster_name': 'docker-cluster', 'cluster_uuid': '2Y-V4oL4SbyoEwrlu-UcBg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [11]:
# Index setting
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/LszLs1lKTOC8oQB16ttAJA] already exists')

In [12]:
from tqdm.auto import tqdm

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [14]:
query = 'I just discovery the course, can I still join now ? '

In [15]:
"""
1. Boost of importance: "fields": ["question^3", "text", "section"]: -> ^3 means 3 times more important than others
2. filter -> similar to where in SQL
"""
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs =[]
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [21]:
results = elastic_search(query)

# 2 - Connect to GenAI API

## prepare the prompt

In [22]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION:{question}

CONTEXT:
{context}
""".strip()
# .strip() to remove leading and trailing whitespace characters from a string — including spaces, tabs (\t), and newline characters (\n).

In [23]:
context = ""

for doc in results: 
    context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

In [24]:
print(context)

section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General

In [26]:
prompt = prompt_template.format(question=query, context=context).strip()

In [27]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION:I just discovery the course, can I still join now ? 

CONTEXT:
section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question

## 1) Gemini AI API
- in terminal : ```pip install -q -U google-genai```
- set environment variable, export GEMINI_API_KEY= "..."

In [28]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text= prompt)])],
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)), # Disables thinking to minimize the cost, otherwise remove this config 
)

print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


## 2) Groq AI API
- set environment variable, export GROQ_API_KEY="..."

In [29]:
from groq import Groq
import os

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

response = client.chat.completions.create(
    messages=[{"role":"user", "content": prompt}], # keyword is "messages"
    model="llama-3.3-70b-versatile",
    stream=False,
)

print(response.choices[0].message.content)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


# 3 - Wrap up and modularze the code

In [30]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'}, # optional
        boost_dict = boost, 
        num_results = 10
    )
    return results

In [31]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION:{question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results: 
        context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [32]:
def llm_gemini(prompt):
    client_gemini = genai.Client()
    response = client_gemini.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            types.Content(
                role="user",
                parts=[types.Part(text= prompt)])],
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0)), # Disables thinking to minimize the cost, otherwise remove this config 
        )

    return response.text

In [33]:
def llm_groq(prompt):
    client_groq = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )
    
    response = client_groq.chat.completions.create(
        messages=[{"role":"user", "content": prompt}], # keyword is "messages"
        model="llama-3.3-70b-versatile",
        stream=False,
    )
    
    return response.choices[0].message.content

In [34]:
# query = "how to run python with kafka?"
# query = "Can I still enroll?"
query = 'I just discovery teh course, can I still join now ? '
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer_gemini = llm_gemini(prompt)
    answer_groq = llm_groq(prompt)
    return (
        "========== GEMINI ANSWER ==========\n"
        f"{answer_gemini}\n\n"
        "========== GROQ ANSWER ==========\n"
        f"{answer_groq}"
    )

In [35]:
rag(query)

"========== GEMINI ANSWER ==========\nYes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\n========== GROQ ANSWER ==========\nYes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [36]:
print(rag(query))

========== GEMINI ANSWER ==========
Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

========== GROQ ANSWER ==========
Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [38]:
# print(rag("the class already started, can I still enroll? "))
print(rag("how to run python with kafka?"))

========== GEMINI ANSWER ==========
NONE

========== GROQ ANSWER ==========
NONE
